In [1]:
import pandas as pd
import datetime
import numpy as np
import sys

sys.path.append("..")
from aoc18.utils import read_input
inp = read_input(4, root_path="..")
inp_test = read_input(4, test=True, root_path="..")

# DAY 4

In [2]:
def split_phrase(phrase):
    splitted = phrase.split('] ')
    return (
        datetime.datetime.strptime(splitted[0][1:], "%Y-%m-%d %H:%M"),
        splitted[1].strip()
    )

def get_guard_id(event):
    return event.split("#")[-1].split(" ")[0]

dts, events = zip(*map(split_phrase, inp))
df = pd.DataFrame({"date": dts, "event": events})\
    .sort_values("date")

dts, events = zip(*map(split_phrase, inp_test))
df_test = pd.DataFrame({"date": dts, "event": events})\
    .sort_values("date")

In [4]:
sorted_events = df.event
sorted_events_test = df_test.event

In [8]:
guards = {}
for df_idx, event in sorted_events.iteritems():
    if "Guard" in event:
        guard_id = get_guard_id(event)
        if guard_id not in guards:
            guards[guard_id] = np.zeros(60)
    elif "falls" in event:
        start = df.loc[df_idx, "date"].minute
    else:
        stop = df.loc[df_idx, "date"].minute
        guards[guard_id][start:stop] = guards[guard_id][start:stop] + 1
        
    

In [9]:
g = np.argmax(map(sum, guards.values()))
gid = int(list(guards.keys())[g])
most_minute_asleep = np.argmax(list(guards.values())[g])
print(gid, most_minute_asleep)

3229 39


In [12]:
guards_test = {}
for df_idx, event in sorted_events_test.iteritems():
    if "Guard" in event:
        guard_id = get_guard_id(event)
        if guard_id not in guards_test:
            guards_test[guard_id] = np.zeros(60)
    elif "falls" in event:
        start = df_test.loc[df_idx, "date"].minute
    else:
        stop = df_test.loc[df_idx, "date"].minute
        guards_test[guard_id][start:stop] = guards_test[guard_id][start:stop] + 1
        
g = np.argmax(map(sum, guards_test.values()))
gid = int(list(guards_test.keys())[g])
most_minute_asleep = np.argmax(list(guards_test.values())[g])
print(guards_test['99'].sum())
print(gid, most_minute_asleep)  

30.0
10 24


In [18]:
guard_ids, hours_asleep = zip(*(map(lambda pair: (int(pair[0]), pair[1].sum()), guards.items())))
my_g_id = guard_ids[np.argmax(hours_asleep)]
my_g_id * guards[str(my_g_id)].argmax()

39584

In [20]:
guard_ids, count, minutes_most_asleep = zip(*(map(lambda pair: (int(pair[0]), pair[1].max(), pair[1].argmax()), guards.items())))
my_g_id = guard_ids[np.argmax(count)]
my_g_id * minutes_most_asleep[np.argmax(count)]

55053

In [21]:
guard_ids, count, minutes_most_asleep = zip(*(map(lambda pair: (int(pair[0]), pair[1].max(), pair[1].argmax()), guards_test.items())))
my_g_id = guard_ids[np.argmax(count)]
my_g_id * minutes_most_asleep[np.argmax(count)]

4455

# DAY 5

In [19]:
inp = read_input(5, split_lines=False, root_path="..").strip()
inp_test = read_input(5, test=True, split_lines=False, root_path="..").strip()

In [77]:
def react(char1, char2):
    return (char1.lower()==char2.lower()) & (char1!=char2)

def check_left_and_right(left, right):
    return len(left)>0 and len(right)>0 and (react(left[-1], right[0]))

def chunk(code):
    if len(code)<=1:
        return code
    elif len(code)==2:
        return "" if react(code[0], code[1]) else code
    else:
        cut = len(code)//2
        left, right = code[:(cut+1)], code[(cut+1):]
        while check_left_and_right(left, right):
            left, right = left[:-1], right[1:]
        chunk_l = chunk(left)
        chunk_r = chunk(right)
        code = chunk_l + chunk_r
        if check_left_and_right(chunk_l, chunk_r):
            return chunk(code)
        else:
            return code

In [78]:
len(chunk(inp))

RecursionError: maximum recursion depth exceeded in comparison

In [101]:
def reduce_code(code):
    i=0
    while i<(len(code)-1):
        if not(react(code[i], code[i+1])):
            i+=1
        else:
            code = code[:i] + code[(i+2):]
    return code

def run1(code):
    red = reduce_code(code)
    while red!=code:
        code = red
        red = reduce_code(code)
    return len(code), code

In [102]:
run1(inp)[0]

9202

In [105]:
def remove_unit(inp, unit):
    return inp.replace(unit, "").replace(unit.upper(), "")
    
    
def run2(inp):
    chunk = run1(inp)[1]
    print("first run done")
    units = set(map(lambda x: x.lower(), inp))
    res = {}
    mini = len(inp)
    c = 0
    for unit in units:
        c+=1
        res[unit] = run1(remove_unit(chunk, unit))[0]
        if res[unit]<mini:
            mini = res[unit]
    print(c)
    return mini

In [106]:
run2(inp)

first run done
26


6394